In [1]:
%load_ext autotime

# Postage Stamp Cube (PSC) Creation

For each of the sources that were detected we want to slice out the Postage Stamp Cube (PSC). The _x_ and _y_ values detected from `sextractor` are used to located the star, however this position is then adjusted such that every stamp in the cube is aligned and has a red-pixel in the (0,0) position. The _x,y_ values for each frame should lie within the center superpixel on the corresponding stamp.

The stamps contain the raw data, i.e. no bias removed or any other processing. 

Each stamp is saved along with the timestamp and picid inside a csv file, one file per PSC.

In [2]:
import os

import numpy as np
import pandas as pd

from tqdm import tqdm_notebook as tqdm

from piaa.utils import pipeline
from piaa.utils import helpers

from astropy.io import fits

time: 2.11 s


In [3]:
# Get detected sources
base_dir = '/var/panoptes/images/fields/PAN012/358d0f/20180822T035809/'
# base_dir = '/var/panoptes/images/fields/PAN001/Hd189733/14d3bd/20180913T085704/'
source_filename = os.path.join(base_dir, f'point-sources-filtered.csv')

# Actually get the sources
sources = pipeline.lookup_sources_for_observation(filename=source_filename).set_index(['picid'], append=True)

# Make directory for PSC
os.makedirs(os.path.join(base_dir, 'stamps'), exist_ok=True)

time: 1.54 s


In [4]:
# Used for progress display
num_sources = len(list(sources.index.levels[1].unique()))

time: 3.59 ms


In [5]:
for pid, target_table in tqdm(sources.groupby('picid'), total=num_sources, desc="Making PSCs"):
    stamps = list()
    for idx, row in target_table.iterrows():
        date_obs= idx[0]
        
        # Get the data for the entire frame
        data = fits.getdata(os.path.join(base_dir, row.file)) 

        # Get the stamp for the target
        target_slice = helpers.get_stamp_slice(row.x, row.y, stamp_size=(stamp_size, stamp_size), ignore_superpixel=False)

        # Get data
        stamps.append(data[target_slice].flatten())
        
    pd.DataFrame(stamps, index=target_table.index).to_csv(os.path.join(base_dir, 'stamps', f'{pid}.csv'))

NameError: name 'stamp_size' is not defined

time: 364 ms
